In [1]:
from scipy.sparse import csr_matrix
import numpy as np
#### Sparse Retrieval using SPARTA #### 
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.sparse import SparseSearch
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir import util
#### Sparse Retrieval using SPARTA #### 
from beir.retrieval import models
import pickle

/opt/anaconda3/envs/rra/lib/python3.12/site-packages/beir/retrieval/search/sparse/sparse_search.py:6: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


In [ ]:
data_set = "scifact"
data_path="evaluation_datasets"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(data_set)
data_path = util.download_and_unzip(url, data_path)
corpus, queries_, qrels_ = GenericDataLoader(data_folder=data_path).load(split="test")
queries_ids = list(queries_.keys())
queries= list(queries_.values())
documents = [[f"{doc['title']} ,{doc['text']}"] for doc in corpus.values()]
document_ids= list(corpus.keys())

In [ ]:
from src.utils import get_data#, get_torch_sparse,get_score
from src.utils.utils import get_torch_sparse,get_score

corpus, queries, qrels  =get_data("scifact")
queries_ids=list(queries.keys())
document_ids= list(corpus.keys())

ImportError: cannot import name 'get_torch_sparse' from 'src.utils' (/Users/arthur/Documents/reasearch/RRA/src/utils/__init__.py)

In [ ]:
from src.rra.retrievers import sparta
sparseModel= sparta.get_model()

In [ ]:
import numpy as np
#please modify SPARTA in  beir.retrieval.models from np.int to int and from np.float to float
encoded_doc,encoded_q=sparta.encode(sparseModel,corpus,queries,queries_ids,qrels)
doc_sparse_torch,_= get_torch_sparse(encoded_doc,device="cpu",keep_all=True)


query: 100%|██████████| 300/300 [00:00<00:00, 2555.56it/s]

{'NDCG@1': 0.50333, 'NDCG@3': 0.55849, 'NDCG@5': 0.57191, 'NDCG@10': 0.59784, 'NDCG@100': 0.63118, 'NDCG@1000': 0.64505}


/Users/arthur/Documents/reasearch/RRA/src/utils/utils.py:255: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  values = torch.tensor(coo.data, dtype=dtype, device=device)


In [ ]:
res,full_res=get_score(doc_sparse_torch,encoded_q,document_ids,queries_ids,qrels,top_k=100)
res

(30522, 5183)


{'NDCG@1': 0.50333,
 'NDCG@10': 0.59784,
 'Recall@1': 0.48611,
 'Recall@10': 0.70428,
 'Recall@100': 0.859}

In [45]:
from src.rra.rsa import RSA
rsa_mat,Vd,Ut= RSA(alpha=2, iterations=1,device="cpu", batch_row=10,classic=True,).run(doc_sparse_torch)
rsa_score,rsa_full_res= get_score(rsa_mat,encoded_q,document_ids,queries_ids,qrels,Vd,Ut,top_k=100)
rsa_score

100%|██████████| 3053/3053 [00:00<00:00, 3295.30it/s]


(30522, 5183)


{'NDCG@1': 0.53,
 'NDCG@10': 0.62402,
 'Recall@1': 0.51278,
 'Recall@10': 0.73178,
 'Recall@100': 0.893}

In [13]:
30522* 5183,10366000/158195526

(158195526, 0.0655265054714632)